In [3]:
from helium import *
from bs4 import BeautifulSoup
import pandas as pd
from time import sleep

In [4]:
driver = start_chrome()

In [5]:
go_to('https://platform.ctf.hkcert.org/scoreboard')

In [14]:
df = pd.DataFrame()

In [15]:
t=[]
c=[]

In [177]:
l1 = Text(to_left_of=Button('Next'),to_right_of=Button('Previous')).value

In [178]:
l1

'Page 1 of 121'

In [160]:
l1[l1.index('Page')+4:l1.index('of')]

' 1 '

In [16]:
def to_page(i):
    sleep(1)
    for j in range(i-1):
        click('next')

In [17]:
def start_page():
    l1 = Text(to_left_of=Button('Next'),to_right_of=Button('Previous')).value
    p =int(l1[l1.index('Page')+4:l1.index('of')])
    for i in range(p-1):
        click("Previous")
    

In [245]:
start_page()

In [18]:
def parse_team(i,j):
    if Text(i).exists():
        click(i)
    elif Text(i.strip()).exists():
        click(i.strip())
    else:
        print("Not found")
        driver.execute_script("window.history.go(-1)")
        sleep(0.2)
        go_to_page(j)
        return {}
    cnt=0
    while Text("No orders").exists():
     sleep(0.5)
     if cnt>3:
        driver.execute_script("window.history.go(-1)")
        sleep(0.2)
        go_to_page(j)
        return {}
     cnt+=1
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    s = [i.text for i in soup.find_all('span')]
    data = s[s.index('Price')+1:s.index('Events')]
    score = data[-1]
    data = data[:-4]
    challs = data[::2]
    points = data[1::2]
    driver.execute_script("window.history.go(-1)")
    sleep(0.2)
    go_to_page(j)
    s1 = {"challenge_name":challs,"challenge_points":points,"challenge_score":score}
    return s1

In [218]:
#from selenium.webdriver.common.by import By
def go_to_page(i):
    sleep(0.2)
    global q
    #global s 
    q = driver.page_source
    if q =='<html><head></head><body></body></html>':
        driver.execute_script("window.history.go(1)")
    soup = BeautifulSoup(q,'html.parser')
    #l1 = Text(to_left_of=Button('Next'),to_right_of=Button('Previous')).value
    l1 = soup.css.select('span[class="px-1.5"]')[0].text
    p =int(l1[l1.index('Page')+4:l1.index('of')])
    #s = driver.find_elements(By.XPATH,'//button[text()="Next"]')[0]
    for i in range(i-p):
        click("Next")
        #s.click()
    sleep(0.1)

In [108]:
q

'<html><head></head><body></body></html>'

In [ ]:
for i in range(51):
    go_to_page(i)
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    teams = [i.text for i in soup.find_all('span')]
    ans = []
    for cnt,j in enumerate(teams):
        if 'Page' in j:
            ans=teams[cnt-10:cnt]
            break
    try:
        for k in ans:
         p = parse_team(k,i)
         sleep(1)
         c.append(p)
         t.append(k)
    except:
        for n in ans[ans.index(k):]:
         go_to_page(i)
         p = parse_team(n,i)
         sleep(1)
         c.append(p)
         t.append(n)

In [214]:
df['Team'] = t

In [215]:
df["Challenges"] = c

In [216]:
df.to_csv('hkcert.csv')